In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/Homework1

**FINAL MODEL CODE**

Here follows the code to build our final model. We chose this model because out of all our models it produced the best score on the hidden test set: 0.9057. We start by importing the following libraries and by setting the random seed, the directories and the names of the classes.

In [ ]:
import tensorflow as tf
import os
import random
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_hub as hub #tensorflow_hub is the hub from which we will fetch our transfer learning model

tfk = tf.keras
tfkl = tf.keras.layers

seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

#the dataset has already been unzipped and imported, 
dataset_dir = 'training'
training_dir = os.path.join('training')
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

DATA AUGMENTATION: we then used ImageDataGenerator to import the images and to apply data augmentation of the dataset. Since an explicit validation set was not provided, we decided to split the data by setting "validation_split" as parameter of ImageDataGenerator. We decided to set it to 0.1, thus obtaining a training set of 15963 elements and a validation set of 1765 elements.

Then we proceeded with data augmentation, knowing from our previous networks that this would surely improve our model. Given the shapes of the leafs and the ways a leaf can appear in the dataset, we decided to set a high rotation factor, a large zoom range and also a brightness range.

Lastly, we normalized both sets.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rotation_range=45, #high rotation factor, given the different ways a leaf could show up in the dataset
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        featurewise_center=True,
                                        zoom_range=[0.5,1.0], #zoom range to grasp the various types of images
                                        shear_range=0.2,
                                        horizontal_flip=True, #both flips true
                                        vertical_flip=True, 
                                        fill_mode='nearest',
                                        channel_shift_range=0.2,
                                        brightness_range = (0.5, 1.5),
                                        rescale=1/255., #normalization
                                        validation_split=0.10) #validation_split se to 0.1


val_data_gen = ImageDataGenerator(validation_split=0.10,
                                     rescale=1/255.,) 

train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed,
                                               subset='training') #name given to the train set
valid_gen = val_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=False,
                                               seed=seed,
                                               subset = 'validation') #name given to the validation set

We then setted the input shape. Given that we want to perform transfer learning, we specify a variable url containing the URL of the model we selected.

In [ ]:
input_shape = (256, 256, 3)
url = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/feature_vector/2'

The URL contains a feature vector. The pourpose of this imported neural network is to extract features from the dataset provided in input. In particular, this model is an EfficientNet V2 trained on the Imagenet dataset (2012 version). 

Once the features have been extracted, we proceed by building a fully connected layer of 14 units. This layer, which will be the last layer of our model, will take the imported EfficientNet V2 in input and it will provide us, in output, the class to which the input image has been assigned to. 

To do so, we defined the function that will build the model.

In [ ]:
def build_model(url, classes): #takes in input the URL and the number of classes
    
    tf_layer = hub.KerasLayer(url,
                             trainable = False, #we set this to False, we want to use the weights trained on Imagenet 
                             name = 'tf_layer',
                             input_shape = input_shape)
    
    model = tf.keras.Sequential([
        tf_layer,
        tfkl.Dense(classes, activation='softmax', name = 'output') #as said before, we connect the imported "layer" to a FC dense layer of 14 neurons (or units) which will be the final output of our model
    ])
    
    return model

Knowing that the fine tuned model will give us a higher accuracy (0.90 compared to the 0.83 accuracy of the non fine tuned one) we immediately proceeded with the declaration of the function that will build the fine tuned model.

The big difference is that now we set the "trainable parameter" to True. In this way, we "unfreeze" the parameters of the imported model and we train the whole model on our dataset.

In [ ]:
def fine_tuned_model(url, classes): 
    
    tf_layer = hub.KerasLayer(url,
                             trainable = True, #we set this to True to unfreeze all the layers and try to learn the weights that were freezed in the transfer learning
                             name = 'tf_layer',
                             input_shape = input_shape)
    
    model = tf.keras.Sequential([
        tf_layer,
        tfkl.Dense(classes, activation='softmax', name = 'output') #as said before, we connect the imported "layer" to a FC dense layer of 14 neurons (or units) which will be the final output of our model
    ])
    
    return model

In [ ]:
ft_model = fine_tuned_model(url, 14)


ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy') #as suggested by the hub, we use the Adam optimizer
ft_model.summary()

From the summary in output, we can observe that the fine tuned model (ft_model) has a total number of 8,706,812 trainable parameters. As expected, this number is way higher than the number of trainable parameters of the transfer learning model (19,726).

We then proceed by fitting the model and training our CNN. 

In [ ]:
history = ft_model.fit(
    x = train_gen, #we assign the train set we created
    epochs = 70,
    validation_data = valid_gen, #we assign the validation set we created
).history


ft_model.save("modello_env2_ft")